In [5]:
!pip install matplotlib==3.2.2 numpy==1.21.6 pandas==1.3.5 plotly==5.5.0 requests==2.28.1 pyalgotrading==2022.9.3 tenacity==8.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.2.0
    Uninstalling tenacity-8.2.0:
      Successfully uninstalled tenacity-8.2.0
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [6]:
import requests
import pandas as pd
import plotly.express as px
import json
import numpy as np
import plotly.graph_objects as go
import pyalgotrading
from pyalgotrading.constants import * 
import pyalgotrading.utils.func as candle


In [7]:
key = "SIS7LVZ9X6ES95CG"

In [8]:
def indicator1(df, timeperiod=5):
    """data_frame must contain columns: (timestamp, close).
    """
    df["indicator"] = df["close"].astype("float").rolling(window=timeperiod).mean()
    return df[[ "timestamp", "indicator" ]]




def find_intersecting_points(data, a, b):
    """This function finds the index of row where lines close_data and indicator_data intersect.
    """
    x = np.argwhere(np.diff(np.sign(a - b))).flatten()
    data["signal"] = "NO_SIGNAL"
    
    for i in x:
        if a.iloc[i] > b.iloc[i]:
            data["signal"].iloc[i+1] = "BUY"
        elif a.iloc[i] <= b.iloc[i]:
            data["signal"].iloc[i+1] = "SELL"
    return data



def plot_stock_graph(data):
    """
    data_frame must contain columns: (timestamp, indicator, close).
    pass the data_frame from which cols to be taken.
    """
    fig = px.line(data, 
                x=data["timestamp"], 
                y=[data.close, data.indicator],
                title="Stock's close_price vs. indicator (moving avg).")
    fig.update_layout(xaxis_title="Date - Hour(24h)", yaxis_title="Price")
    fig.show()
  

In [9]:
class ScriptData:
    
    
    def __init__(self,):
        self._scripts = {}
        self.base_ep = "https://www.alphavantage.co/support/SIS7LVZ9X6ES95CG"
        self.function = "TIME_SERIES_INTRADAY"
        self.interval = "60min" 
        
    def fetch_intraday_data(self, script):
        url = f'https://www.alphavantage.co/query?function={self.function}&symbol={script}&interval={self.interval}&apikey={key}'
        response = requests.get(url)
        data = response.json()
        data = data
        self._scripts[script] = data
        
    def convert_intraday_data(self, script):
        if script in self._scripts:
            data = self._scripts[script]
            df = pd.DataFrame(data['Time Series (60min)']).transpose().rename_axis('timestamp').reset_index()
            
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            df = df.sort_values("timestamp", ascending= True)
            df.columns = df.columns.str.replace('[^a-zA-Z]', '', regex=True)
            self._scripts[script] = df
            
        else:
            return None, "Fetch data first."
        

                                         
    def __getitem__(self, script):
        if script in self._scripts:
            return self._scripts[script]
        else:
            return None, "fetch script first."

        
        
    def __contains__(self, script):
        if script in self._scripts:
            return True
        else:
            False
    


In [10]:
class Strategy:
    def __init__(self, script):
        self.script = script
        self._strategies = {}
        self.s = ScriptData()
        
        
        
    def get_script_data(self):
        data = self.s.fetch_intraday_data(self.script)
        data = self.s.convert_intraday_data(self.script)
        data = self.s[self.script]
        self._strategies[self.script] = data

    
    
    def get_signals(self):
        if self.script in self._strategies:
            data = self._strategies[self.script]
            df = indicator1(data, 5).rename(columns={"indicator": "indicator_data"})
            df = df.assign(close_data=data["close"])
            df[["close_data", "indicator_data"]] = df[["close_data", "indicator_data"]].astype("float")
            
            df_ = find_intersecting_points(df, df.close_data, df.indicator_data)
            
            df_signals = df_[(df_["signal"] != "NO_SIGNAL")]
            df_signals = df_signals[["timestamp", "signal"]]
            return df_signals 


    def get_lines_to_plot(self):
        if self.script in self._strategies:
            data = self._strategies[self.script]
            df = indicator1(data, 5)
            df = df.assign(close=data["close"])  
            df[["close", "indicator"]] = df[["close", "indicator"]].astype("float")
            return df

In [11]:
script_data = ScriptData()

In [12]:
# Note: dataframe is in ascending order of dates. The recent ones last and older ones first.
script_data.fetch_intraday_data("GOOGL")
script_data.convert_intraday_data("GOOGL")
script_data["GOOGL"]

,timestamp,open,high,low,close,volume
99,2023-02-07 17:00:00,107.6400,107.9800,107.0126,107.2800,1394922
98,2023-02-07 18:00:00,107.3400,107.6400,106.7500,106.9900,233253
97,2023-02-07 19:00:00,107.0000,107.1200,106.5000,106.8000,72708
96,2023-02-07 20:00:00,106.8000,106.9000,106.5000,106.7700,40249
95,2023-02-08 05:00:00,107.0900,107.1300,106.4200,106.6900,52646
...,...,...,...,...,...,...
4,2023-02-15 16:00:00,96.6900,96.9700,96.3800,96.9700,8602688
3,2023-02-15 17:00:00,96.9400,97.0800,96.8500,96.9500,987263
2,2023-02-15 18:00:00,96.9406,96.9600,96.8600,96.9000,61057
1,2023-02-15 19:00:00,96.9000,97.0100,96.8900,96.9400,68819


In [13]:
# Note: dataframe is in ascending order of dates. The recent ones last and older ones first.
script_data.fetch_intraday_data("NVDA")
script_data.convert_intraday_data("NVDA")
script_data["NVDA"]

,timestamp,open,high,low,close,volume
99,2023-02-07 17:00:00,221.7300,221.8900,220.9237,221.2900,714474
98,2023-02-07 18:00:00,221.2000,221.7300,221.1513,221.6613,61238
97,2023-02-07 19:00:00,221.7500,223.5000,221.7000,222.5000,108176
96,2023-02-07 20:00:00,222.5000,223.5000,222.5000,223.3100,83039
95,2023-02-08 05:00:00,222.5000,224.6000,222.5000,224.2000,57330
...,...,...,...,...,...,...
4,2023-02-15 16:00:00,226.4950,227.7500,225.5400,227.6400,5336904
3,2023-02-15 17:00:00,227.6400,228.0000,227.2800,227.3688,494654
2,2023-02-15 18:00:00,227.3900,227.6120,227.2900,227.3900,18836
1,2023-02-15 19:00:00,227.4000,227.7500,227.1700,227.6300,19781


In [14]:
# Note: dataframe is in ascending order of dates. The recent ones last and older ones first.
script_data.fetch_intraday_data("AAPL")
script_data.convert_intraday_data("AAPL")

In [15]:
script_data["AAPL"]

,timestamp,open,high,low,close,volume
99,2023-02-07 17:00:00,154.3849,154.4248,153.6360,153.7558,1382257
98,2023-02-07 18:00:00,153.7658,153.9655,153.7558,153.9555,52367
97,2023-02-07 19:00:00,153.9156,154.3649,153.8856,153.9654,103632
96,2023-02-07 20:00:00,153.9555,154.0154,153.6759,153.9156,59336
95,2023-02-08 05:00:00,153.9156,154.1452,153.3165,153.4463,75941
...,...,...,...,...,...,...
4,2023-02-15 16:00:00,155.0200,155.3800,154.6700,155.3600,7694429
3,2023-02-15 17:00:00,155.3200,155.4500,155.0800,155.3000,1266637
2,2023-02-15 18:00:00,155.2900,155.3300,155.2404,155.2900,43602
1,2023-02-15 19:00:00,155.2600,155.3900,155.2600,155.3300,33801


In [16]:
"GOOGL" in script_data

True

In [17]:
"AAPL" in script_data

True

In [18]:
"TATA" in script_data

False

In [19]:
"IBM" in script_data

False

In [20]:
"TSLA" in script_data

False

In [21]:
indicator1(script_data["GOOGL"], timeperiod=5)

,timestamp,indicator
99,2023-02-07 17:00:00,NaN
98,2023-02-07 18:00:00,NaN
97,2023-02-07 19:00:00,NaN
96,2023-02-07 20:00:00,NaN
95,2023-02-08 05:00:00,106.90600
...,...,...
4,2023-02-15 16:00:00,96.53268
3,2023-02-15 17:00:00,96.67796
2,2023-02-15 18:00:00,96.82438
1,2023-02-15 19:00:00,96.88838


In [22]:
indicator1(script_data["AAPL"], timeperiod=5)

,timestamp,indicator
99,2023-02-07 17:00:00,NaN
98,2023-02-07 18:00:00,NaN
97,2023-02-07 19:00:00,NaN
96,2023-02-07 20:00:00,NaN
95,2023-02-08 05:00:00,153.80772
...,...,...
4,2023-02-15 16:00:00,154.94200
3,2023-02-15 17:00:00,154.95200
2,2023-02-15 18:00:00,155.19600
1,2023-02-15 19:00:00,155.26000


In [23]:
indicator1(script_data["NVDA"], timeperiod=5)

,timestamp,indicator
99,2023-02-07 17:00:00,NaN
98,2023-02-07 18:00:00,NaN
97,2023-02-07 19:00:00,NaN
96,2023-02-07 20:00:00,NaN
95,2023-02-08 05:00:00,222.59226
...,...,...
4,2023-02-15 16:00:00,226.42068
3,2023-02-15 17:00:00,226.77044
2,2023-02-15 18:00:00,227.06774
1,2023-02-15 19:00:00,227.30376


In [24]:
strategy_nvda = Strategy("NVDA")

In [25]:
strategy_nvda.get_script_data()

In [26]:
strategy_nvda.get_signals()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,timestamp,signal
91,2023-02-08 09:00:00,BUY
90,2023-02-08 10:00:00,SELL
89,2023-02-08 11:00:00,BUY
87,2023-02-08 13:00:00,SELL
86,2023-02-08 14:00:00,BUY
80,2023-02-08 20:00:00,SELL
76,2023-02-09 08:00:00,BUY
75,2023-02-09 09:00:00,SELL
71,2023-02-09 13:00:00,BUY
59,2023-02-10 09:00:00,SELL


In [27]:
strategy_tsla = Strategy("TSLA")
strategy_tsla.get_script_data()
strategy_tsla.get_signals()

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,timestamp,signal
93,2023-02-08 07:00:00,SELL
75,2023-02-09 09:00:00,BUY
74,2023-02-09 10:00:00,SELL
70,2023-02-09 14:00:00,BUY
60,2023-02-10 08:00:00,SELL
58,2023-02-10 10:00:00,BUY
53,2023-02-10 15:00:00,SELL
51,2023-02-10 17:00:00,BUY
47,2023-02-13 05:00:00,SELL
44,2023-02-13 08:00:00,BUY


In [28]:
# plot graph for any script present in scripts. If script not present then fetch and get script_data else it will error out.

# here GOOGL already in script_data
# x axis-> date:month time.

candle.plot_candlestick_chart(script_data["GOOGL"], 
                              PlotType.JAPANESE,  
                              hide_missing_dates=True, 
                              show=True, 
                              plot_height=500, 
                              plot_width=1000,
                             )

In [29]:
"NVDA" in script_data

True

In [30]:
# plot graph for any script present in scripts. If script not present then fetch and get script_data else it will error out.

# NVDA present in script_data.

candle.plot_candlestick_chart(script_data["NVDA"], 
                              PlotType.JAPANESE,  
                              hide_missing_dates=True, 
                              show=True, 
                              plot_height=500, 
                              plot_width=1000)

In [31]:
# optional: plots close and indicator lines of script if existing in the strategies.
# NVDA already present
nvda_df = strategy_nvda.get_lines_to_plot()
plot_stock_graph(nvda_df)